In [1]:
%load_ext autoreload
%autoreload 2
!export PYTHONPATH=$PYTHONPATH:/Users/arvindagarwal/Desktop/cmu/courses/ml/lda2vec-pytorch/utils


In [2]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from tqdm import tqdm
import spacy
from gensim import corpora, models
import pickle
import pandas as pd
from pprint import pprint
import csv 

import sys
sys.path.insert(0,'.')
sys.path.insert(0,'/Users/arvindagarwal/Desktop/cmu/courses/ml/lda2vec-pytorch/utils')
# from utils import preprocess, get_windows
# from utils.preprocess import preprocess
from preprocess_mod import *
from get_windows_mod import *

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['via'])
import re


paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [3]:
!ls ..

20newsgroups                   semeval2016-task6-domaincorpus
LICENSE                        stance
README.md                      utils
loss.png


In [26]:
MIN_COUNTS = 10
MAX_COUNTS = 10000
# words with count < MIN_COUNTS
# and count > MAX_COUNTS
# will be removed

MIN_LENGTH = 3
# minimum document length 
# (number of words)
# after preprocessing

# half the size of the context around a word
HALF_WINDOW_SIZE = 1
# it must be that 2*HALF_WINDOW_SIZE < MIN_LENGTH

# Load NLP model

In [27]:
nlp = spacy.load('en')

# Load dataset

In [28]:
with open('slangs.csv', mode='r') as infile:
    reader = csv.reader(infile) 
    slang_words = {rows[0]:rows[1] for rows in reader}

In [29]:
def clean_tweets(sent):
    sent = str(sent)
    
    # Substitute Slangs
    for word in sent.split(" "):
        if word in slang_words.keys():
            sent = re.sub(word, slang_words[word], sent)
            
    # Remove new line characters
    sent = re.sub('\s+', ' ', sent)

    # Remove http:// links
    sent = re.sub('http:\/\/.*','', sent)

    # Remove https:// links
    sent = re.sub('https:\/\/.*','', sent)
    
    # Remove distracting single quotes
    sent = re.sub("\'", "", sent)

    # Remove distracting single quotes
    sent = re.sub("\"", "", sent)

    # Remove hashtags
    sent = re.sub("\#", "", sent)
    
    return sent

In [30]:
# dataset = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
# docs = dataset['data']

In [31]:
PATH_LABELLED_DATA_TRUMP = "../semeval2016-task6-domaincorpus/data-all-annotations/testdata-taskB-all-annotations.txt"
PATH_UNLABELLED_DATA_TRUMP = "./../semeval2016-task6-domaincorpus/downloaded_Donald_Trump.txt"

In [32]:
X_train = pd.read_pickle('X_train.pkl')
X_test = pd.read_pickle('X_test.pkl')
y_train = pd.read_pickle('y_train.pkl')
y_test = pd.read_pickle('y_test.pkl')

data_labelled_train = pd.concat([X_train, y_train], ignore_index=True, axis=1)
data_labelled_train.columns = ['Tweet', 'Stance']
data_labelled_test = pd.concat([X_test, y_test], ignore_index=True, axis=1)
data_labelled_test.columns = ['Tweet', 'Stance']

In [33]:
data_unlabelled = pd.read_csv(PATH_UNLABELLED_DATA_TRUMP, sep='\t', lineterminator='\n', encoding ='latin1', names = ["ID", "Tweet"])
data_unlabelled = data_unlabelled.where(data_unlabelled.Tweet != 'Not Available')
data_unlabelled.dropna(how='any', inplace=True)
data_unlabelled['Tweet'] = data_unlabelled['Tweet'].apply(lambda x: x[1:])
data_unlabelled['Tweet'] = data_unlabelled['Tweet'].apply(clean_tweets)

In [34]:
combined_data = pd.concat([data_labelled_test, data_labelled_train, data_unlabelled], ignore_index=True)

/Users/arvindagarwal/anaconda3/envs/mlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [35]:
combined_data.count()

ID        53616
Stance      897
Tweet     54513
dtype: int64

In [36]:
combined_data.head(10)

,ID,Stance,Tweet
0,NaN,AGAINST,We need a man that will do the tough negotiati...
1,NaN,AGAINST,"WE Love Mexicans,WE Respect your Work Ethic,Yo..."
2,NaN,NONE,ExtremistProgressives are so Focused on their ...
3,NaN,AGAINST,@BraveConWarrior stories like this make think ...
4,NaN,FAVOR,GOP candidate predictions? America decides to ...
5,NaN,NONE,"...And of course, God bless America!! tcot tea..."
6,NaN,FAVOR,Donald Trump for president? I can dig it. SemST
7,NaN,NONE,@ericbolling :). Sorry for the kidding but had...
8,NaN,NONE,Muslim community chips in to help rebuild burn...
9,NaN,FAVOR,This country needs a President to take control...


In [37]:
df = combined_data['Tweet']

In [38]:
data = df.values.tolist()
docs = data
pprint(data[:5])

['We need a man that will do the tough negotiations, say what needs to be said '
 'and forget the words politically correct. thanks SemST',
 'WE Love Mexicans,WE Respect your Work Ethic,Your Love of Family,Your '
 'Loyalty,Your FOOD &Your Love Of God! Lets ALL Get JOBS,VOTE SemST',
 'ExtremistProgressives are so Focused on their Agenda that they believe a '
 'Inanimate FlagKills but MultitimeDeporteesDont SemST',
 '@BraveConWarrior stories like this make think PATRIOTS are at the end of the '
 'string. all kinds of shit is going to hit fan SemST',
 'GOP candidate predictions? America decides to play the Trump card and go '
 'with the man with the plan. SemST']


In [39]:
# number of documents
len(docs)

54513

In [40]:
# store an index with a document
docs = [(i, doc) for i, doc in enumerate(docs)]

# Preprocess dataset and create windows

In [41]:
encoded_docs, decoder, word_counts = preprocess(
    docs, nlp, MIN_LENGTH, MIN_COUNTS, MAX_COUNTS
)

100%|██████████| 54513/54513 [10:39<00:00, 85.26it/s] 


number of removed short documents: 3322
total number of tokens: 384508
number of tokens to be removed: 86443
number of additionally removed short documents: 4433
total number of tokens: 290462

minimum word count number: 4
this number can be less than MIN_COUNTS because of document removal


In [42]:
# new ids will be created for the documents.
# create a way of restoring initial ids:
doc_decoder = {i: doc_id for i, (doc_id, doc) in enumerate(encoded_docs)}

In [43]:
data = []
# new ids are created here
for index, (_, doc) in tqdm(enumerate(encoded_docs)):
    windows = get_windows(doc, HALF_WINDOW_SIZE)
    # index represents id of a document, 
    # windows is a list of (word, window around this word),
    # where word is in the document
    data += [[index, w[0]] + w[1] for w in windows]

data = np.array(data, dtype='int64')

46758it [00:00, 53545.42it/s]


In [44]:
# a row in 'data' contains:
# id of a document, id of a word in this document, a window around this word
# 1 + 1 + 10
data.shape[1]

4

In [45]:
# number of windows (equals to the total number of tokens)
data.shape[0]

290462

# Get unigram distribution

In [46]:
word_counts = np.array(word_counts)
unigram_distribution = word_counts/sum(word_counts)

# Prepare word vectors

In [51]:
print(decoder)

{0: 'Trump', 1: 'Donald', 2: 'amp', 3: 'like', 4: 'GOP', 5: 'trump', 6: 'donaldtrump', 7: 'say', 8: 'NBC', 9: 'think', 10: 'vote', 11: 'people', 12: 'America', 13: 'need', 14: 'want', 15: 'go', 16: 'get', 17: 'know', 18: 'illegal', 19: 'Mexico', 20: 'good', 21: 'racist', 22: 'immigrant', 23: 'right', 24: 'mexican', 25: 'fire', 26: 'comment', 27: 'run', 28: 'Mexicans', 29: 'time', 30: 'USA', 31: 'DonaldTrumps', 32: 'love', 33: 'talk', 34: 'candidate', 35: 'immigration', 36: 'come', 37: 'tell', 38: 'look', 39: 'president', 40: 'win', 41: 'Macys', 42: 'speak', 43: 'man', 44: 'poll', 45: 'country', 46: 'word', 47: 'tcot', 48: 'SemST', 49: 'great', 50: 'truth', 51: 'support', 52: 'thing', 53: 'tie', 54: 'watch', 55: 'republican', 56: 'let', 57: 'cut', 58: 'thank', 59: 'Univision', 60: 'bad', 61: 'campaign', 62: 'President', 63: 'Miss', 64: 'job', 65: 'dump', 66: 'take', 67: 'call', 68: 'make', 69: 'new', 70: 'business', 71: 'way', 72: 'ElChapo', 73: 'remark', 74: 'border', 75: 'stop', 76: '

In [58]:
print(encoded_docs)

[(0, [13, 43, 723, 13, 7, 299, 46, 650, 374, 58, 48]), (1, [2248, 28, 293, 32, 1435, 1010, 32, 344, 56, 64, 10, 48]), (2, [881, 118, 48]), (3, [369, 3, 9, 1047, 129, 381, 109, 15, 263, 480, 48]), (4, [4, 34, 3502, 12, 621, 250, 0, 351, 43, 236, 48]), (5, [243, 344, 1468, 12, 47, 917, 48]), (6, [1, 0, 39, 1111, 48]), (8, [2691, 810, 151, 2692, 811, 1408, 2349]), (9, [45, 13, 62, 385, 57, 42, 50, 48]), (11, [338, 236, 1762, 422, 67, 21, 12, 370, 167, 48]), (12, [1160, 327, 3118, 21, 1409, 3118, 1409, 1265, 48]), (13, [54, 95, 2955, 362, 397, 60, 1617, 713, 48]), (14, [160, 116, 78, 269, 456, 5, 643, 754, 48]), (15, [16, 1932, 319, 0, 394, 48]), (16, [208, 1143, 0, 356, 38, 90, 976, 21, 276, 363, 48]), (17, [2459, 643, 1211, 918, 319, 38, 2956, 319, 48]), (18, [1, 0, 338, 1144, 48]), (19, [1, 0, 300, 3279, 148, 118, 1811, 515, 538, 48]), (20, [1618, 2957, 48]), (21, [23, 1578, 76, 27, 45, 92, 48]), (22, [208, 1, 0, 43, 110, 45, 525, 48]), (23, [2693, 691, 48]), (24, [5, 276, 790, 743, 216

In [53]:
# %%time
vocab_size = len(decoder)
embedding_dim = 50

# train a skip-gram word2vec model
texts = [[str(j) for j in doc] for i, doc in encoded_docs]
model = models.Word2Vec(texts, size=embedding_dim, window=3, workers=4, sg=1, negative=15, iter=70)
model.init_sims(replace=True)

In [59]:
word_vectors = np.zeros((vocab_size, embedding_dim)).astype('float32')
for i in decoder:
#     print(str(i))
    if str(i) == '3894':
        continue
    word_vectors[i] = model.wv[str(i)]

In [60]:
# number of unique words
vocab_size

3895

# Prepare initialization for document weights

In [61]:
texts = [[decoder[j] for j in doc] for i, doc in encoded_docs]
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [62]:
# %%time
n_topics = 25
lda = models.LdaModel(corpus, alpha=0.9, id2word=dictionary, num_topics=n_topics)
corpus_lda = lda[corpus]

In [63]:
for i, topics in lda.show_topics(n_topics, formatted=False):
    print('topic', i, ':', ' '.join([t for t, _ in topics]))

topic 0 : Trump GOP Donald laugh Lord loud amp bear trump love
topic 1 : Trump GOP Donald vote donaldtrump trump say think amp immigration
topic 2 : Trump Donald trump Cruz talk amp say Campaign like GOP
topic 3 : Trump trump donaldtrump Donald like say give amp think GOP
topic 4 : military Trump Donald like amp GOP say trump America need
topic 5 : Trump Donald elect look amp get trump say flag like
topic 6 : GOP Trump UniteBIue Donald amp say trump Plug Party get
topic 7 : Trump tcot Donald ccot poll obama pjnet think good amp
topic 8 : like worth Policy straight Trump net amp vote Donald gop
topic 9 : Trump America Donald amp come say like figure trump GOP
topic 10 : Trump Time Donald like amp say tell trump Lol tweet
topic 11 : Trump amp GOP list Donald donaldtrump need say illegal like
topic 12 : Trump death Donald like people amp say hour GOP China
topic 13 : Iran Trump prison say like Hannity donaldtrump amp reveal trump
topic 14 : Trump touch threaten party Donald amp Poll GOP d

In [64]:
doc_weights_init = np.zeros((len(corpus_lda), n_topics))
for i in tqdm(range(len(corpus_lda))):
    topics = corpus_lda[i]
    for j, prob in topics:
        doc_weights_init[i, j] = prob

100%|██████████| 46758/46758 [00:14<00:00, 3207.11it/s]


# Save data

In [65]:
np.save('data.npy', data)
np.save('word_vectors.npy', word_vectors)
np.save('unigram_distribution.npy', unigram_distribution)
np.save('decoder.npy', decoder)
np.save('doc_decoder.npy', doc_decoder)
np.save('doc_weights_init.npy', doc_weights_init)